In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    FFF_Train_Decile7 = pd.read_pickle("FFF_Train_Decile7.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile7.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile7, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 7')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 12s 4ms/step - loss: 0.3458 - acc: 0.8684 - val_loss: 0.2942 - val_acc: 0.8887
Epoch 36/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3301 - acc: 0.8700 - val_loss: 0.2760 - val_acc: 0.9005
Epoch 54/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.3311 - acc: 0.8678 - val_loss: 0.2755 - val_acc: 0.9018
Epoch 55/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3304 - acc: 0.8668 - val_loss: 0.2785 - val_acc: 0.8940
Epoch 56/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3273 - acc: 0.8714 - val_loss: 0.2718 - val_acc: 0.9018
Epoch 57/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3252 - acc: 0.8766 - val_loss: 0.2732 - val_acc: 0.9031
Epoch 58/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3172 - acc: 0.8704 - val_loss: 0.2726 - val_acc: 0.9018
Epoch 59/75
3055/3055 [==============================] - 14s

3055/3055 [==============================] - 12s 4ms/step - loss: 0.2924 - acc: 0.8782 - val_loss: 0.2638 - val_acc: 0.9058
Epoch 70/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2905 - acc: 0.8795 - val_loss: 0.2638 - val_acc: 0.9058
Epoch 71/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2932 - acc: 0.8717 - val_loss: 0.2637 - val_acc: 0.9058
Epoch 72/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2863 - acc: 0.8812 - val_loss: 0.2638 - val_acc: 0.9058
Epoch 73/75
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2873 - acc: 0.8776 - val_loss: 0.2637 - val_acc: 0.9058
Epoch 74/75
3055/3055 [==============================] - 8s 3ms/step - loss: 0.2924 - acc: 0.8750 - val_loss: 0.2636 - val_acc: 0.9058
Epoch 75/75
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2873 - acc: 0.8809 - val_loss: 0.2635 - val_acc: 0.9058
Test accuracy: 0.905759161367
Train on 3055 samples, validate

3055/3055 [==============================] - 12s 4ms/step - loss: 0.3009 - acc: 0.8743 - val_loss: 0.2609 - val_acc: 0.9045
Epoch 91/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3025 - acc: 0.8727 - val_loss: 0.2608 - val_acc: 0.9045
Epoch 92/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3066 - acc: 0.8746 - val_loss: 0.2608 - val_acc: 0.9045
Epoch 93/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2944 - acc: 0.8691 - val_loss: 0.2609 - val_acc: 0.9045
Epoch 94/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2967 - acc: 0.8756 - val_loss: 0.2608 - val_acc: 0.9045
Epoch 95/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3533 - acc: 0.8615 - val_loss: 0.3093 - val_acc: 0.8809
Epoch 9/75
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3506 - acc: 0.8625 - val_loss: 0.3023 - val_acc: 0.8848
Epoch 10/75
3055/3055 [==============================] -

3055/3055 [==============================] - 13s 4ms/step - loss: 0.2926 - acc: 0.8753 - val_loss: 0.2675 - val_acc: 0.8992
Epoch 30/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2931 - acc: 0.8812 - val_loss: 0.2668 - val_acc: 0.8992
Epoch 31/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2909 - acc: 0.8746 - val_loss: 0.2670 - val_acc: 0.8979
Epoch 32/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2933 - acc: 0.8743 - val_loss: 0.2670 - val_acc: 0.8992
Epoch 33/125
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2984 - acc: 0.8668 - val_loss: 0.2673 - val_acc: 0.8992
Epoch 34/125
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2908 - acc: 0.8720 - val_loss: 0.2671 - val_acc: 0.8992
Epoch 35/125
3055/3055 [==============================] - 14s 4ms/step - loss: 0.2918 - acc: 0.8717 - val_loss: 0.2670 - val_acc: 0.8992
Epoch 36/125
3055/3055 [==============================

...

3055/3055 [==============================] - 12s 4ms/step - loss: 0.3632 - acc: 0.8625 - val_loss: 0.3138 - val_acc: 0.8783
Epoch 28/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3524 - acc: 0.8694 - val_loss: 0.3123 - val_acc: 0.8770
Epoch 29/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3534 - acc: 0.8704 - val_loss: 0.3080 - val_acc: 0.8822
Epoch 30/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3493 - acc: 0.8723 - val_loss: 0.3057 - val_acc: 0.8861
Epoch 31/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3496 - acc: 0.8664 - val_loss: 0.3025 - val_acc: 0.8887
Epoch 32/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3410 - acc: 0.8753 - val_loss: 0.3037 - val_acc: 0.8848
Epoch 33/125
3055/3055 [==============================] - 11s 4ms/step - loss: 0.3477 - acc: 0.8707 - val_loss: 0.2983 - val_acc: 0.8901
Epoch 34/125
3055/3055 [==============================

3055/3055 [==============================] - 12s 4ms/step - loss: 0.3038 - acc: 0.8746 - val_loss: 0.2652 - val_acc: 0.9058
Epoch 98/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3116 - acc: 0.8743 - val_loss: 0.2653 - val_acc: 0.9058
Epoch 99/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3062 - acc: 0.8743 - val_loss: 0.2651 - val_acc: 0.9058
Epoch 100/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3064 - acc: 0.8759 - val_loss: 0.2651 - val_acc: 0.9071
Epoch 101/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3056 - acc: 0.8759 - val_loss: 0.2651 - val_acc: 0.9058
Epoch 102/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3053 - acc: 0.8786 - val_loss: 0.2652 - val_acc: 0.9058
Epoch 103/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3028 - acc: 0.8773 - val_loss: 0.2650 - val_acc: 0.9058
Epoch 104/125
3055/3055 [=========================

3055/3055 [==============================] - 13s 4ms/step - loss: 0.2849 - acc: 0.8792 - val_loss: 0.2589 - val_acc: 0.9110

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 31/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2860 - acc: 0.8759 - val_loss: 0.2588 - val_acc: 0.9110
Epoch 32/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2851 - acc: 0.8759 - val_loss: 0.2590 - val_acc: 0.9110
Epoch 33/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2816 - acc: 0.8763 - val_loss: 0.2590 - val_acc: 0.9097
Epoch 34/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2889 - acc: 0.8733 - val_loss: 0.2592 - val_acc: 0.9110
Epoch 35/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2821 - acc: 0.8763 - val_loss: 0.2591 - val_acc: 0.9110
Epoch 36/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2871 - acc: 0.8845 - val_loss: 0.2592 - val_a

3055/3055 [==============================] - 12s 4ms/step - loss: 0.2850 - acc: 0.8773 - val_loss: 0.2587 - val_acc: 0.9110
Epoch 90/100
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2814 - acc: 0.8822 - val_loss: 0.2590 - val_acc: 0.9071
Epoch 91/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2898 - acc: 0.8812 - val_loss: 0.2589 - val_acc: 0.9071
Epoch 92/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2868 - acc: 0.8786 - val_loss: 0.2591 - val_acc: 0.9058
Epoch 93/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2848 - acc: 0.8792 - val_loss: 0.2586 - val_acc: 0.9110
Epoch 94/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2787 - acc: 0.8812 - val_loss: 0.2587 - val_acc: 0.9110
Epoch 95/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2856 - acc: 0.8773 - val_loss: 0.2587 - val_acc: 0.9110
Epoch 96/100
3055/3055 [==============================

3055/3055 [==============================] - 12s 4ms/step - loss: 0.3034 - acc: 0.8789 - val_loss: 0.2664 - val_acc: 0.9018
Epoch 48/50
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3065 - acc: 0.8746 - val_loss: 0.2664 - val_acc: 0.9018
Epoch 49/50
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3107 - acc: 0.8746 - val_loss: 0.2663 - val_acc: 0.9031
Epoch 50/50
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3083 - acc: 0.8750 - val_loss: 0.2666 - val_acc: 0.9018
Test accuracy: 0.901832459797
Evalutation of best performing model:
1504/1504 [==============================] - 1s 969us/step
[0.26959690578440404, 0.89494680851063835]
Best performing model chosen hyper-parameters:
{'epochs': 2, 'lr': 2}
Confusion Matrix for LSTM predictions:
[[ 282  105]
 [  53 1064]]
Classification Report for LSTM predictions:
-------------------------------------------
             precision    recall  f1-score   support

          0       0.

In [ ]:
0.93484